In [1]:
# ======================================================================
# ⚙️ [연구 4단계] 경쟁 페르소나 분석
# ======================================================================
# 목표: 3단계에서 분류된 주제(Topic)들을 각 가게가 얼마나 차지하고 있는지,
#      '주제별 시장 점유율'을 계산하여 경쟁 구도를 명확한 표로 만듭니다.
# ======================================================================

# 1. 라이브러리 임포트 및 분석 결과 로드
import pandas as pd
from bertopic import BERTopic
import os

try:
    # 3단계에서 생성된, 각 리뷰에 토픽 번호가 할당된 데이터를 불러옵니다.
    df = pd.read_csv('../data/reviews_with_topics.csv')
    
    # 3단계에서 생성된, 키워드가 수정된 BERTopic 모델을 불러옵니다.
    topic_model = BERTopic.load("../models/bertopic_model_final")
    
    print("----- 데이터 및 토픽 모델 불러오기 성공 -----")
except FileNotFoundError:
    print("오류: 이전 단계의 결과 파일을 찾을 수 없습니다.")
    print("03_topic_modeling_final.ipynb를 먼저 실행했는지 확인하세요.")
    exit()


# 2. 주제별 가게 점유율 계산
print("\n----- 주제별 가게 점유율 분석 시작 -----")

# -1 토픽 (이상치)은 분석에서 제외합니다.
df_filtered = df[df['topic'] != -1].copy()

# 'topic'과 'store_id'를 기준으로 리뷰 개수를 집계합니다.
topic_store_counts = df_filtered.groupby(['topic', 'store_id']).size().reset_index(name='review_count')

# 각 주제(topic) 내에서 가게별 점유율(%)을 계산합니다.
topic_total_counts = topic_store_counts.groupby('topic')['review_count'].sum().reset_index(name='total_count')
analysis_df = pd.merge(topic_store_counts, topic_total_counts, on='topic')
analysis_df['share(%)'] = round((analysis_df['review_count'] / analysis_df['total_count']) * 100, 1)

# 보기 쉽게 피벗 테이블(Pivot Table) 형태로 변환합니다.
market_share_pivot = analysis_df.pivot_table(index='topic', columns='store_id', values='share(%)').fillna(0)


# 3. 최종 분석 테이블 생성
# 토픽 모델에서 토픽별 정보를 가져옵니다. (이제 키워드가 제대로 나옵니다)
topic_info = topic_model.get_topic_info()
topic_info = topic_info[topic_info['Topic'] != -1]

# 최종 분석 결과 테이블을 위해, 토픽 정보와 시장 점유율 정보를 합칩니다.
final_analysis_table = pd.merge(topic_info[['Topic', 'Count', 'CustomName']], 
                                market_share_pivot, 
                                left_on='Topic', 
                                right_index=True).rename(columns={"CustomName": "Keywords"})

# 💡 [핵심 수정] 논문을 위해 가게 이름을 'Store A', 'Store B', 'Store C'로 통칭합니다.
store_names = {'A': 'Store A', 'B': 'Store B', 'C': 'Store C'}
final_analysis_table = final_analysis_table.rename(columns=store_names)

# 논문에 직접 사용할 최종 테이블을 정렬합니다.
thesis_table = final_analysis_table[['Topic', 'Count', 'Keywords', 'Store A', 'Store B', 'Store C']].sort_values(by='Topic').reset_index(drop=True)


# 4. 최종 결과 출력 및 저장
print("\n\n===== [논문 결과 1] 최종 경쟁 페르소나 분석 테이블 =====")
display(thesis_table)

if not os.path.exists("../results"): os.makedirs("../results")
thesis_table.to_csv('../results/competitive_analysis_final.csv', index=False, encoding='utf-8-sig')

print("\n\n✅ 경쟁 분석이 완료되어 '../results/competitive_analysis_final.csv' 파일로 저장했습니다.")



2025-09-25 22:10:11,356 - BERTopic - WARNING: You are loading a BERTopic model without explicitly defining an embedding model. If you want to also load in an embedding model, make sure to use `BERTopic.load(my_model, embedding_model=my_embedding_model)`.


----- 데이터 및 토픽 모델 불러오기 성공 -----

----- 주제별 가게 점유율 분석 시작 -----


===== [논문 결과 1] 최종 경쟁 페르소나 분석 테이블 =====


,Topic,Count,Keywords,Store A,Store B,Store C
0,0,30,"구이, 일본, 볶음밥, 범계, 웨이팅",10.0,6.7,83.3
1,1,16,"이자카야, 범계, 이태원, 술집, 사장",31.2,68.8,0.0
2,2,13,"친절, 직원, 음식, 감사, 최고",0.0,69.2,30.8
3,3,11,"범계, 술집, 안주, 분위기, 맛집",90.9,9.1,0.0
4,4,8,"안주, 오늘, 분위기, 만족, 저녁",0.0,37.5,62.5
5,5,7,"범계, 술집, 꼬치, 모찌리도후, 맛집",100.0,0.0,0.0
6,6,4,"친구, 웨이팅, 대기, 시간, 남자",0.0,0.0,100.0
7,7,3,"친구, 안주, 범계, 술집, 최고",33.3,66.7,0.0




✅ 경쟁 분석이 완료되어 '../results/competitive_analysis_final.csv' 파일로 저장했습니다.
